# Notebook to extract variables out of NHD database


In [24]:
# ee env

Originally 2021.02.23_ExportImagestoGCS_MR1_Copy.ipynb

In [25]:
2

2

# Authentications

In [26]:
import pandas as pd
import numpy as np
import time
import ee
import pickle
ee.Initialize()

# Patch Size

In [27]:
PATCH_SIZE = 100 # 100 or 1250

# Set up bounding box

In [28]:
def square(lat, lon, size):
  crs_proj = "EPSG:4326"  
  return ee.Geometry.Point([lon, lat], proj=crs_proj).buffer(size).bounds()

In [29]:
full_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

no_data_states = ["LA", "MD", "SC"]

states = list(set(full_states) - set(no_data_states))

# NWI

In [30]:
merged = ee.FeatureCollection([None])

state_dict = {"AK" : ["North", "Central", "South"],
             "CA" : ["North", "NorthCentral", "South", "SouthCentral"],
             "WI" : ["North", "South"],
             "CO" : ["East", "West"],
             "IA" : ["East", "West"],
             "KS" : ["East", "West"],
             "MN" : ["Central_West", "East", "North_Central", "North_East", "North_West", "South"],
             "MT" : ["East", "West"],
             "ND" : ["East", "West"],
             "NE" : ["East", "West"],
             "NV" : ["North", "South"],
             "OK" : ["East", "West"],
             "OR" : ["East", "West"],
             "SD" : ["East", "West"],
             "TX" : ["East", "West", "Central"],
             "WI" : ["North", "South"],
             "WY" : ["East", "West"]}

for state in states:
    directions_list = state_dict.get(state)
    if directions_list == None:
        merged = merged.merge(ee.FeatureCollection("users/madhukarreddy/NWI_" + state +"_Wetlands"))
    else:
        for direction in directions_list:
            merged = (merged.merge(ee.FeatureCollection("users/madhukarreddy/NWI_" + 
                                                        state +  
                                                        "_Wetlands" + "_" + 
                                                        direction )))
fc = merged

In [31]:
# fc.getInfo()

In [32]:
# fc_getInfo = fc.filterBounds(square(lat=45.475649, lon=-69.471018 , size=PATCH_SIZE)).getInfo()
# fc_getInfo = fc_getInfo.get('features')
# num_of_features = len(fc_getInfo)
# print("number of features =", num_of_features)

# attr_list = []
# wetland_list = []
# for feature in range(num_of_features):
#     # some features are described as "PSS3/EM1E"
#     # split them out so each attritube can be joined to the wetland defns csv
#     attr_element = fc_getInfo[feature].get('properties').get('ATTRIBUTE').split("/")
#     [attr_list.append(element) for element in attr_element]
# #     attr_list.append(fc_getInfo[feature].get('properties').get('ATTRIBUTE'))

#     wetland_element = fc_getInfo[feature].get('properties').get('WETLAND_TY').split("/")
#     [wetland_list.append(element) for element in wetland_element]
# #     wetland_list.append(fc_getInfo[feature].get('properties').get('WETLAND_TY'))


# print(attr_list)
# print(wetland_list)

In [33]:
def nwi_bbox_features(lat=45.475649, lon=-69.471018 , size=PATCH_SIZE):
    """
    input: lat, lon, and size=half of bbox side
    output:
        - num_of_features: number of NWI features intersected by bbox
        - attr_list: list of NWI attributes for wetland types
        - wetland_list: WETLAND_TY for each ATTRIBUTE
    """
    try:
        fc_getInfo = fc.filterBounds(square(lat, lon, size)).getInfo()
        fc_getInfo = fc_getInfo.get('features')
        num_of_features = len(fc_getInfo)
        print("number of features =", num_of_features)

        attr_list = []
        wetland_list = []
        for feature in range(num_of_features):

            # some features are described as "PSS3/EM1E"
            # split them out so each attritube can be joined to the wetland defns csv
            attr_element = fc_getInfo[feature].get('properties').get('ATTRIBUTE').split("/")
            [attr_list.append(element) for element in attr_element]

            wetland_element = fc_getInfo[feature].get('properties').get('WETLAND_TY').split("/")
            [wetland_list.append(element) for element in wetland_element]
        return num_of_features, attr_list, wetland_list

    except Exception as e:
        print(e)
        print("Issue with at lat={}, lon={}".format(lat, lon))
        return np.nan
    

nwi_bbox_features()                                 

number of features = 4


(4,
 ['R5UBH', 'PUBH', 'PEM1E', 'PSS3', 'EM1E'],
 ['Riverine',
  'Freshwater Pond',
  'Freshwater Emergent Wetland',
  'Freshwater Forested',
  'Shrub Wetland'])

In [34]:
# read in csv file with SSURGO variables
# df_m = pd.read_csv("combined_regular_clean_with_ssurgo_variables.csv")
df_m = pd.read_pickle("cwr_nwpr_dataset")

In [35]:
def extract_nwi_bbox_variables(df_m_, i, size):

  df_m_["nwi_bbox_vars_" + str(2 * PATCH_SIZE) + "m"] = (df_m_.apply(lambda x: 
                                                     nwi_bbox_features(lat=x.latitude, 
                                                                       lon=x.longitude, 
                                                                       size=size), axis=1))


#   pickle.dump(df_m_, open("NWI_extracted_vars_200mX200m/200mX200m_nwi_variables_part" + str(i),"wb"), protocol=3)        
  (pickle.dump(df_m_, open("NWI_extracted_vars_cwr_nwpr_" + str(2 * PATCH_SIZE) + "m/nwi_extraction_" 
                           + str(2 * PATCH_SIZE) + "X" + str(2 * PATCH_SIZE) + "m"
                           + "_part" + str(i),"wb"), 
               protocol=3))        

In [36]:
from datetime import datetime
print(datetime.now())

2021-04-02 22:14:30.272683


In [37]:
df_m.shape[0]//50

64

In [38]:
# pass in batches of 500
# MADHUKAR: 0 - 5000 
# SHOBHA: 5000 - 10000
# RADHIKA: 10000 - 15000

batch_size = 50
MY_NAME = "MADHUKAR"
START = 0 + 5000 * (MY_NAME == "SHOBHA") + 10000 * (MY_NAME == "RADHIKA")

# for batch in range(10, df_m.shape[0]//50):
for batch in [64]:
  print("batch {} of 10 started".format(batch + 1))
  batch_df = df_m[START + batch_size * batch : START + batch_size * (batch + 1)].copy()
  extract_nwi_bbox_variables(batch_df, (START + batch_size * batch) + 1, size = PATCH_SIZE)
  print("batch {} of 10 done".format(batch + 1))


batch 65 of 10 started
number of features = 2
number of features = 3
number of features = 0
number of features = 3
number of features = 0
number of features = 1
number of features = 2
number of features = 2
number of features = 1
number of features = 0
number of features = 0
number of features = 2
number of features = 1
number of features = 2
number of features = 2
number of features = 1
number of features = 4
number of features = 3
batch 65 of 10 done


In [17]:
# # pass in batches of 500
# # MADHUKAR: 0 - 5000 
# # SHOBHA: 5000 - 10000
# # RADHIKA: 10000 - 15000

# batch_size = 500
# MY_NAME = "SHOBHA"
# START = 0 + 5000 * (MY_NAME == "SHOBHA") + 10000 * (MY_NAME == "RADHIKA")

# for batch in range(10):
#   print("batch {} of 10 started".format(batch + 1))
#   batch_df = df_m[START + batch_size * batch : START + batch_size * (batch + 1)].copy()
#   extract_nwi_bbox_variables(batch_df, (START + batch_size * batch) + 1, size = PATCH_SIZE)
#   print("batch {} of 10 done".format(batch + 1))


In [18]:
# # pass in batches of 500
# # MADHUKAR: 0 - 5000 
# # SHOBHA: 5000 - 10000
# # RADHIKA: 10000 - 15000

# batch_size = 500
# MY_NAME = "RADHIKA"
# START = 0 + 5000 * (MY_NAME == "SHOBHA") + 10000 * (MY_NAME == "RADHIKA")

# for batch in range(10):
#   print("batch {} of 10 started".format(batch + 1))
#   batch_df = df_m[START + batch_size * batch : START + batch_size * (batch + 1)].copy()
#   extract_nwi_bbox_variables(batch_df, (START + batch_size * batch) + 1, size = PATCH_SIZE)
#   print("batch {} of 10 done".format(batch + 1))


In [19]:
from datetime import datetime
print(datetime.now())

2021-04-02 21:02:12.285928


In [20]:
#read the file
# df_m_test = pd.read_csv('NHD_extracted_vars/combined_regular_clean_with_ssurgo_nhd_variables_part1.csv')
df_m_test = pd.read_pickle("NWI_extracted_vars_cwr_nwpr_" + str(2 * PATCH_SIZE) + "m/nwi_extraction_" 
                           + str(2 * PATCH_SIZE) + "X" + str(2 * PATCH_SIZE) + "m"
                           + "_part1")



In [21]:
df_m_test["nwi_bbox_vars_" + str(2 * PATCH_SIZE) + "m"]

14324                                          (0, [], [])
14339                                          (0, [], [])
14346    (16, [R1UBV, R1UBV, R3UBH, R5UBH, PEM1R, PEM1R...
14348                                          (0, [], [])
14378    (2, [PSS3Dg, PSS3Dg], [Freshwater Forested, Sh...
14395    (2, [PUBHx, PUBHx], [Freshwater Pond, Freshwat...
14540                             (1, [R4SBC], [Riverine])
14604                                          (0, [], [])
14609                                          (0, [], [])
14659    (4, [R4SBC, R5UBH, PUBH, PEM1Ad], [Riverine, R...
14668                                          (0, [], [])
14680    (10, [PUB, ABFx, PUB, ABFx, PUB, ABFx, PUB, AB...
14691         (1, [PEM1Cd], [Freshwater Emergent Wetland])
14693                                          (0, [], [])
14696    (3, [R5UBFx, PABHx, PEM1A], [Riverine, Freshwa...
14723                                          (0, [], [])
14726                                          (0, [], [

In [22]:
df_m_test.columns

Index(['jurisdiction_type', 'da_number', 'latitude', 'longitude',
       'cwa_determination', 'Index', 'nwi_bbox_vars_200m'],
      dtype='object')

In [23]:
PATCH_SIZE


100